In [ ]:
!pip install --force-reinstall "chronos-forecasting[training] @ git+https://github.com/zepengli99/chronos-forecasting.git"

In [1]:
!pip install "chronos-forecasting[training] @ git+https://github.com/zepengli99/chronos-forecasting.git"

  Cloning https://github.com/zepengli99/chronos-forecasting.git to /tmp/pip-install-bpku11uv/chronos-forecasting_4fd4765ed3134d0f8046ba9dff115dac
  Running command git clone --filter=blob:none --quiet https://github.com/zepengli99/chronos-forecasting.git /tmp/pip-install-bpku11uv/chronos-forecasting_4fd4765ed3134d0f8046ba9dff115dac
  Resolved https://github.com/zepengli99/chronos-forecasting.git to commit 57d052e96a17dbfee5df957ec8bdfc47beb581ef
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/NLP_project/chronos-forecasting')

# Fintune chronos-t5-tiny model

In [5]:
%%time
!CUDA_VISIBLE_DEVICES=0 python scripts/training/train.py --config scripts/training/configs/chronos-t5-tiny-btc.yaml \
    --model-id amazon/chronos-t5-tiny \
    --no-random-init \
    --max-steps 1000 \
    --learning-rate 0.001

2025-02-07 19:16:23.406542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738955783.736039    1848 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738955783.826653    1848 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-07 19:16:24.561749: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-07 19:16:32,997 - /content/drive/MyDrive/NLP_project/chronos-forecasting/scripts/training/train.py - INFO - T

# Upload fine-tuned chronos-t5-tiny model (optional)

In [ ]:
# import os
# os.environ["HF_TOKEN"] = Your hugging face token


In [ ]:
# from chronos import ChronosPipeline

# pipeline = ChronosPipeline.from_pretrained("./output/run-0/checkpoint-final/")
# pipeline.model.model.push_to_hub("zepengli99/chronos-t5-tiny-btc")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zepengli99/chronos-t5-tiny-btc/commit/d1ee926ad4bf93ba477078debef9231cd9b0bc69', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='d1ee926ad4bf93ba477078debef9231cd9b0bc69', pr_url=None, repo_url=RepoUrl('https://huggingface.co/zepengli99/chronos-t5-tiny-btc', endpoint='https://huggingface.co', repo_type='model', repo_id='zepengli99/chronos-t5-tiny-btc'), pr_revision=None, pr_num=None)

# Evaluation

In [6]:
!python scripts/evaluation/evaluate.py scripts/evaluation/configs/btc.yaml scripts/evaluation/results/chronos-t5-tiny-btc-finetuned.csv \
    --chronos-model-id "./output/run-3/checkpoint-final/" \
    --batch-size=32 \
    --device=cuda:0 \
    --num-samples 20

2025-02-07 19:32:15.767466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738956735.790680    6692 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738956735.797616    6692 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-07 19:32:15.820524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-07 19:32:20,049 - Chronos Evaluation - INFO - Loading bitcoin_test
Generating test split: 100% 1/1 [00:00<00:

In [3]:
import pandas as pd

df = pd.read_csv('scripts/evaluation/results/chronos-t5-tiny-btc-finetuned.csv')
df

,dataset,model,MASE,WQL
0,bitcoin_test,./output/run-3/checkpoint-final/,0.173046,0.003901
